## Library

In [ ]:
from confusion import *

In [ ]:
from my_models import *

In [ ]:
from geneval import *

In [ ]:
model, tokenizer = my_pythia()

In [ ]:
def scan_prompts(xs, my_prompt, model, tokenizer,
                      n_iter = 8, out_length = 10, temperature = 1.):
    '''
    generate text for a parametrized prompt.
    xs: parameters
    my_prompt: a parametrizeable prompt
    
    returns dict with key = parameter, value = list of generated texts
    '''
    ys = {j: [] for j in xs}
    for j in xs:
        prompt = my_prompt(j)
        generated_output = f_generate_batch(model, tokenizer, [prompt]*n_iter, out_length,
                                            temperature = temperature)
        print(j, generated_output)
        ys[j] = (generated_output)
    return ys

In [ ]:
def logprobs_prompts(model, tokenizer, my_prompt, ys, xs, dxs):
    '''
    return logprobs for prompts scan
    '''
    res = {}
    for j in xs:
        for jj in dxs:
            print(j,jj)
            prompt = my_prompt(j+jj)
            f_eval_temp = f_eval_prob_batch(model, tokenizer, [prompt]*len(ys[j]), ys[j])
            for jjj, y_temp in enumerate(f_eval_temp):
                res[j,j+jj, jjj] = y_temp
    return res

## Program

### prompt scan

In [ ]:
def my_prompt(k):
    return f'{2020+k}'

In [ ]:
xs = list(range(-15,16,1))
dxs = list(range(-1,2,1))
n_iter = 2056

xs, dxs

In [ ]:
ys = scan_prompts(xs, my_prompt, model, tokenizer)

In [ ]:
res = logprobs_prompts(model, tokenizer, my_prompt, ys, xs, dxs)

In [ ]:
# calculate dissimilarity g with L-parameter L
L=1
Y, dY = stat_indicator_scan_final(xs,ys,res,L, g = lambda x: 2*x-1)

In [ ]:
# plot
import pylab as plt
plt.plot(0.5*(np.array(xs[L:])+np.array(xs[:-L])), Y, 'o--')
plt.xlabel('integer')
plt.ylabel('dissimilarity')